In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import time
import json
import pandas as pd


In [2]:
options = Options()
options.headless = False 
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
options.add_argument("disable-infobars")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

### Mantel


In [4]:
url = "https://www.zara.com/de/de/damen-jacken-maentel-l1184.html?v1=2419032"

driver.get(url)
time.sleep(5)

wait = WebDriverWait(driver, 10)

time.sleep(3)

def gradual_scroll(driver, pause_time, step_size):
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        for i in range(0, last_height, step_size):
            driver.execute_script(f"window.scrollTo(0, {i});")
            time.sleep(pause_time)
        
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        last_height = new_height

# Hacer scroll gradual con pausas y pasos pequeños
gradual_scroll(driver, pause_time=0.2, step_size=50)

# Espera adicional para asegurarse de que todo el contenido ha cargado
time.sleep(5)

# Extraer el HTML de la página
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# Cerrar el navegador
driver.quit()

# Guardar el HTML en un archivo para análisis posterior
#with open("zara_mujer_blazer.html", "w", encoding='utf-8') as file:
    #file.write(page_source)

print("El scroll, el cambio de vista y la extracción han sido completados.")

products = soup.find_all('li', class_='product-grid-product')

category = "neuware"
sex = "damen"


data = {
    "Category":[],
    "Sex": [], 
    "Product": [],
    "Color": [],
    "Subcategory": [],
    "Reference": [],
    "Price": [],
    "Image": [],
    "Link": []
}

for product in products:
    # Extraer el nombre y color del producto desde el atributo 'alt' de <img>
    image_tag = product.find('img', class_='media-image__image')
    if image_tag and 'alt' in image_tag.attrs:
        alt_text = image_tag['alt']
        # Verificar si se puede dividir en dos partes: nombre y color
        split_text = alt_text.split(' - ')
        if len(split_text) == 2:
            name, color = split_text
        else:
            name = alt_text  # Si no hay un guion, tomamos todo como el nombre
            color = "No Color"  # Color por defecto
    else:
        name = "No Name"
        color = "No Color"
    
    # Extraer el enlace al producto desde el <a> con clase 'product-link'
    link = product.find('a', class_='product-link')['href'] if product.find('a', class_='product-link') else "No Link"
    
    # Extraer la subcategoría desde el link, basándonos en palabras clave
    subcategory = link.split('/')[-1].split('-')[0]  # Primera palabra de la URL del producto (ej. "capa")

    # Extraer el precio del producto desde el <span> con clase 'money-amount__main'
    price = product.find('span', class_='money-amount__main').get_text(strip=True) if product.find('span', class_='money-amount__main') else "No Price"
    
    # Extraer la referencia del producto (viene como 'data-productid' en la etiqueta <li>)
    reference = product.get('data-productid', 'No Reference')
    
    # Extraer la imagen del producto desde el <img> con clase 'media-image__image'
    image = product.find('img', class_='media-image__image')['src'] if product.find('img', class_='media-image__image') else "No Image"
    
    #category = category

    #sex = sex 

    # Guardar la información en el diccionario
    data["Category"].append(category)
    data["Sex"].append(sex)
    data["Product"].append(name)
    data["Color"].append(color)
    data["Subcategory"].append(subcategory)
    data["Reference"].append(reference)
    data["Price"].append(price)
    data["Image"].append(image)
    data["Link"].append(link)

# Convertir el diccionario en un DataFrame de pandas
df = pd.DataFrame(data)

df.to_csv("../01_data/df_zara_mantel.csv", index= True)

In [14]:
#df

,Category,Sex,Product,Color,Subcategory,Reference,Price,Image,Link
0,neuware,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun / Braungrau von Zara,langer,410888372,"89,95 EUR",https://static.zara.net/assets/public/a9e5/f05...,https://www.zara.com/de/de/langer-wendemantel-...
1,neuware,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun / Braungrau von Zara,langer,410774601,"89,95 EUR",https://static.zara.net/assets/public/0a65/b2f...,https://www.zara.com/de/de/langer-wendemantel-...
2,neuware,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun / Braungrau von Zara,langer,410888372,"89,95 EUR",https://static.zara.net/assets/public/fe16/c55...,https://www.zara.com/de/de/langer-wendemantel-...
3,neuware,damen,LANGER WENDEMANTEL MIT DOPPELTEM GESICHT,Braun / Braungrau von Zara,langer,410888372,"89,95 EUR",https://static.zara.net/assets/public/868d/7ce...,https://www.zara.com/de/de/langer-wendemantel-...
4,neuware,damen,STRICKMANTEL MIT AUFGESETZTEN TASCHEN,Grau meliert von Zara,strickmantel,385398845,"69,95 EUR",https://static.zara.net/assets/public/0aa6/a50...,https://www.zara.com/de/de/strickmantel-mit-au...
...,...,...,...,...,...,...,...,...,...
172,neuware,damen,PRK FLD 13,Cremefarben von Zara,prk,372776391,"99,95 EUR",https://static.zara.net/assets/public/e76b/967...,https://www.zara.com/de/de/prk-fld-13-p0832825...
173,neuware,damen,STEPPJACKE MIT KAPUZE WATER AND WIND PROTECTION,Schwarz von Zara,steppjacke,388982381,"59,95 EUR",https://static.zara.net/assets/public/18b3/8be...,https://www.zara.com/de/de/steppjacke-mit-kapu...
174,neuware,damen,OVERSIZE-CAPETRENCHCOAT ZW COLLECTION,Toffeefarben von Zara,oversize,404602355,"149,00 EUR",https://static.zara.net/assets/public/6374/191...,https://www.zara.com/de/de/oversize-capetrench...
175,neuware,damen,OVERSIZE-CAPETRENCHCOAT ZW COLLECTION,Toffeefarben von Zara,oversize,406075276,"149,00 EUR",https://static.zara.net/assets/public/1a41/0df...,https://www.zara.com/de/de/oversize-capetrench...


In [102]:
#df.to_csv("../01_data/df_zara_mantel.csv", index= True)

In [3]:
options = Options()
options.headless = False 
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
options.add_argument("disable-infobars")


def scrape_zara_data(urls, categories):
    # Initialize the driver (Make sure to set up your driver path accordingly)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)  # Set the path to your ChromeDriver

    # Data storage
    data = {
        "Category": [],
        "Sex": [],
        "Product": [],
        "Color": [],
        "Subcategory": [],
        "Reference": [],
        "Price": [],
        "Image": [],
        "Link": []
    }

    # Iterate over both URLs and Categories
    for url, category in zip(urls, categories):
        # Open the URL
        driver.get(url)
        time.sleep(5)

        # Scroll the page gradually
        def gradual_scroll(driver, pause_time, step_size):
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                for i in range(0, last_height, step_size):
                    driver.execute_script(f"window.scrollTo(0, {i});")
                    time.sleep(pause_time)
                time.sleep(pause_time)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

        gradual_scroll(driver, pause_time=0.2, step_size=50)
        
        # Additional wait to ensure content loads
        time.sleep(5)

        # Extract the HTML of the page
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract product data
        products = soup.find_all('li', class_='product-grid-product')

        sex = "damen"  # Assuming sex is constant in this case, you can modify it as needed

        for product in products:
            image_tag = product.find('img', class_='media-image__image')
            if image_tag and 'alt' in image_tag.attrs:
                alt_text = image_tag['alt']
                split_text = alt_text.split(' - ')
                if len(split_text) == 2:
                    name, color = split_text
                else:
                    name = alt_text
                    color = "No Color"
            else:
                name = "No Name"
                color = "No Color"

            link = product.find('a', class_='product-link')['href'] if product.find('a', class_='product-link') else "No Link"
            subcategory = link.split('/')[-1].split('-')[0]
            price = product.find('span', class_='money-amount__main').get_text(strip=True) if product.find('span', class_='money-amount__main') else "No Price"
            reference = product.get('data-productid', 'No Reference')
            image = product.find('img', class_='media-image__image')['src'] if product.find('img', class_='media-image__image') else "No Image"

            data["Category"].append(category)  # Use the dynamic category
            data["Sex"].append(sex)
            data["Product"].append(name)
            data["Color"].append(color)
            data["Subcategory"].append(subcategory)
            data["Reference"].append(reference)
            data["Price"].append(price)
            data["Image"].append(image)
            data["Link"].append(link)

    # Close the browser
    driver.quit()

    # Convert the data to a pandas DataFrame
    df_zara = pd.DataFrame(data)
    
    # Save the DataFrame as a CSV file
    df_zara.to_csv("../01_data/df_zara.csv", index=True)

    print("Scraping completed and data saved to CSV.")

# Example usage with multiple URLs and corresponding categories:
urls = [
    "https://www.zara.com/de/de/damen-jacken-maentel-l1184.html?v1=2419032",
    "https://www.zara.com/de/de/damen-jacken-l1114.html?v1=2417772",
    "https://www.zara.com/de/de/damen-blazer-l1055.html?v1=2420942",
    "https://www.zara.com/de/de/damen-kleider-l1066.html?v1=2420896",
    "https://www.zara.com/de/de/damen-hosen-l1335.html?v1=2420795",
    "https://www.zara.com/de/de/damen-t-shirts-l1362.html?v1=2420417"
    "https://www.zara.com/de/de/damen-roecke-l1299.html?v1=2420454",
    "https://www.zara.com/de/de/damen-schuhe-l1251.html?v1=2419160",
    "https://www.zara.com/de/de/damen-taschen-l1024.html?v1=2417728",
    "https://www.zara.com/de/de/damen-lingerie-l4021.html?v1=2419807",
    "https://www.zara.com/de/de/damen-schonheit-parfums-l1415.html?v1=2419833",
    "https://www.zara.com/de/de/woman-beauty-makeup-l4414.html?v1=2418919"

    # Add more URLs as needed
]

categories = [
    "maentel",
    "jacken",
    "blazer",
    "kleider",
    "hosen",
    "t_shirts",
    "roecke",
    "schuhe",
    "taschen",
    "lingerie",
    "parfums",
    "make_up"


    # Add corresponding categories as needed
]

scrape_zara_data(urls, categories)

Scraping completed and data saved to CSV.
